# Lab 2 - Statistikk

## CSV-filer
I denne oppgaven skal du gjøre statistikk på dataen fra andre labøvelse. Det er lagt ved eksempeldata i CSV-filen (CSV står for *comma seperated values*) og ser slik ut:
```
"m_Zn / g","V_\mathrm{gass / mL","P_0 / mmHg","t / degC","h_vann / mm"
0.090,34.4,1.013,23.2,135
0.092,35.1,0.9926,22,133
...
```

"`,`" i CSV-filen er skilletegnet mellom celler så teksten over tilsvarer følgende tabell: 
$$
\begin{array}{lllll}
m_\mathrm{Zn}\,/\,\mathrm g & V_\mathrm{gass}\,/\,\mathrm{mL} & P_0\,/\,\mathrm{mmHg} & t\,/\,\mathrm{^\circ C} & h_\mathrm{vann}\,/\,\mathrm{mm}\\
\hline
0.090 & 34.4 & 1.013 & 23.2 & 135 \\
0.092 & 35.1 & 0.9926 & 22 & 133  \\
\vdots & \vdots & \vdots & \vdots & \vdots
\end{array}
$$

CSV-filer er veldig enkle å lese for programmer fordi de er så enkelt skrevet (sammenlignet med f.eks. xlsx-filer fra Microsoft Excel). 

## Dine data

<div class="alert alert-warning">
Det er anbefalt at du også lagrer filene dine i et CSV-format eller lignende. Hvis du har dataene allerede i et regneprogram som Excel eller LibreOffice Calc er det lett å eksportere dem til .csv med *Lagre som*-funksjonen.
</div>

## Lese CSV med NumPy
Under er det et eksempel på hvordan man kan lese en CSV-fil med NumPy. Funksjonen `np.genfromtxt` kan lett konvertere tekstfiler til NumPy-arrayer. Det første argumentet er filnavnet. For å lese filen slik på ønsket måte må man legge til noen instillinger:
- `delimiter`-variabelen sier hvilke(t) tegn som skiller verdiene. For CSV er dette gjerne "`,`" (det er jo tross alt *comma* serperated values), men kan også være andre ting som mellomrom.
- `skip_header`-variabelen er hvor mange linjer på starten av filen som ikke er selve verdiene. I eksempelfilen er det én linje med navn på hver kolonne med data.
- `unpack`-variablen er om vi vil ha arrays med kolonnene (`unpack=True`) eller radene (`unpack=False` *standard*).

<div class="alert alert-danger">
Pass godt på enhetene til verdiene du bruker. Python vet ikke hvilke enheter variabler er i så du må sørge for at alle beregninger skjer i kompatible enheter. I denne labben er det anbefalt å endre alle verdier for 
<ul>
  <li>masse til gram</li>
  <li>volum til liter</li>
  <li>trykk til atmosfærer</li>
  <li>temperatur til Kelvin</li>
</ul> 
</div>

In [ ]:
# Extract variables from data source using NumPy
import numpy as np
m, V_gas, P_0, t, h_H2O = np.genfromtxt('example_data.csv', delimiter=',', skip_header=1, unpack=True)
# Convert degrees Celsius to Kelvin and mL to L
T = t + 273.15
V_gas = V_gas / 1000

# Print absolute temperature to check if values were extracted correcly
print(T)

## Beregn molar masse

<div class="alert alert-warning">
De neste delen tar for seg prelabben, men med programmering. Det kan være lurt å ha den fullførte prelabben forran seg når man gjør disse oppgavene. 
</div>

Du skal i slutten av denne oppgaven ha laget en funksjon som regner ut molarmassen til metallet. Men det er lurt å dele beregningene opp i flere mindre funksjoner — da blir det mye mer oversiktlig!

### Vannets damptrykk
I prelabben gir Tabell 2 vannets damptrykk ved forskjellige temperaturer og du må bruke formelen
$$P(t)=\frac{P_1-P_2}{t_1-t_2}\left(t-t_2\right)+P_2$$
til lineær interpolasjon.

NumPy har denne formelen innebygget. Den heter `interp(t, ti, Pi)` der `t` er temperaturen man vil interpolore for, `ti` og `Pi` er arrays med henholdsvis de kjente temperaturene og damptrykkene. (`interp` fungere så klart for andre ting en temperatur og trykk, men det er disse størrelse vi er interessert i.) Under er et eksempel på bruk av av funksjonen med visualisering.

In [ ]:
# Known values
x = [0, 3, 4, 7, 9]
y = [2, 15, 18, 19, 19]
# Interpolate
x_int = np.linspace(0, 9, 30)
y_int = np.interp(x_int, x, y)

# Visualise interpolation
import matplotlib.pyplot as plt
plt.plot(x, y, 'o', label='known values')
plt.plot(x_int, y_int, '.', label='interpolated values')
plt.legend()
plt.show()

Filen `vapour_pressure_water.csv` inneholder samme data som Tabell 2 i prelabben og ser slik ut:
```
"t / degC","P_vanndamp / atm"
16,0.01793
18,0.02036
...
```
Bruk funksjonen `np.genfromtxt` til å lese dataene fra CSV-filen (se [delen om å lese CSV-filer](#Lese-CSV-med-NumPy)) og bruk `np.interp` til å lage funksjonen `P_vap_H2O(T)` som gir vannets damptrykk ved en (absolutt) temperatur `T`.

<div class="alert alert-danger">
Husk å sjekke enheter og konvertere der det er nødvendig. 
</div>

In [ ]:
def P_vap_H2O(T):
    ### YOUR ANSWER ###

Du har allerede sett en del testfunksjoner med `assert`. Pythons innebygde `assert` egner seg dårlig for arrays, men NumPy har en klasse med funksjoner kalt `numpy.testing` som gjør det kjapt å skrive testfunksjoner for arrays. 

Under brukes testfunksjonen `assert_almost_equal(actual, expected, decimal=d)` som gir `AssertionError` hvis arrayene er forskjellig inntil det desimal nummer `d`. Hvis testen feiler vises arrayene så det er lett å se hvor de er forskjellige.

In [ ]:
import numpy.testing as npt
npt.assert_almost_equal(P_vap_H2O([297.5, 302.7, 298.6, 299.1, 294.2]), 
                                  [.0301, .0408, .0321, .0331, .0247], decimal=4)

### Vanntrykket
Lag en funksjon `P_H2O(h_H2O)` som regner ut vanntrykket i atmosfærer gitt vannhøyden `h_H2O` i millimeter. I koden er det allerede noen konstanter som kan være nyttige.

In [ ]:
def P_H2O(h_H2O):
    d_H2O = 1.00
    d_Hg = 13.6
    mmHg_per_atm = 760. 
    ### YOUR ANSWER ###

Koden under tester ```P_H2O```. Kjør den etter du er ferdig med å implementere ```P_H2O```.

In [ ]:
npt.assert_almost_equal(P_H2O(np.array([93.8, 25.6, 190.8, 9.4, 118.])), 
                                       [.0091, .0025, .0185, .0009, .0114], decimal=4)

### Hydrogentrykket
Lag en funksjon `P_H2(P_H2O, P_0, T)` som regner ut hydrogentrykket i atmosfærer gitt vanntrykket `P_H2O` i atmosfærer, barometertrykket `P_0` i atmosfærer og temperature `T` i Kelvin. 

In [ ]:
def P_H2(P_H2O, P_0, T):
    ### YOUR ANSWER ###

### Sett funksjonene sammen for å beregne molarmasse
Funksjonen `molar_mass(h, V, T, m)` kan lages nå. `h` er vannhøyden i millimeter, `V` er det målte volumet i liter, `T` er den (absolutte) temperaturen og `m` er massen til metallet. Gasskonstanten `R` er allerede oppgitt i passende enheter.

In [ ]:
def molar_mass(h, V, T, m):
    R = 8.2057338e-2 # L atm / K mol
    ### YOUR ANSWER ###

Cellen under burde nå printe fornuftige verdier for molarmassen til metallet.

In [ ]:
molar_mass(h_H2O, V_gas, T, m)

## Konfidensintervall


Konfidensintervall er et intervall som en gitt andel av gangene vil inneholde den sanne verdien av konstant. Antar man at målingene er normalfordelte, er det $p$-konfidensintervallet gitt ved
$$\left(\bar x-t_p\frac s{\sqrt{n}}, \bar x+t_p\frac s{\sqrt{n}}\right)$$
der $\bar x$ er gjennomsnittet, $s$ er standardavviket, $n$ er antall obserbasjoner og $t_p$ er en konstant bestemt av $p$.

Skriv en funksjon `conf_int` som returnerer de to grensene til intervallet. Kode for å regne ut konstanten $t_p$ er allerede oppgitt.

In [ ]:
def conf_int(data, p=.95):
    from scipy.stats import t
    n = np.size(data)
    tp = t.cdf(p, n - 1)
    ### YOUR ANSWER ###

Bruk `conf_int` på verdiene du fikk for molarmassen. Er litteraturverdien for molarmassen i intervallet? Hva forteller svaret deg?

Hvis litteraturverdien ikke er innenfor, er det greit? Kan du evt. forklare hvorfor den ikke er innenfor?